<div style="color:white;display:fill;border-radius:8px;
            background-color:#2a2a2ad9;font-size:190%;
            font-family:Segoe UI;letter-spacing:0.5px">
    <p style="padding: 8px
;color:white;"><b>MACHINE LEARNIG PREDICTION & COHORT ANALYSIS - HOTEL IN LISBON </b></p>
</div>. 

**A cohort** is a group of subjects who share a defining characteristic. We can observe how a cohort behaves across time and compare it to other cohorts. Cohorts are used in medicine, psychology, econometrics, ecology and many other areas to perform a cross-section (compare difference across subjects) at intervals through time.

**Types of cohorts**:

* **Time Cohorts** are customers who signed up for a product or service during a particular time frame. Analyzing these cohorts shows the customers’ behavior depending on the time they started using the company’s products or services. The time may be monthly or quarterly even daily.
* **Behaviour cohorts** are customers who purchased a product or subscribed to a service in the past. It groups customers by the type of product or service they signed up. Customers who signed up for basic level services might have different needs than those who signed up for advanced services. Understaning the needs of the various cohorts can help a company design custom-made services or products for particular segments.
* **Size cohorts** refer to the various sizes of customers who purchase company’s products or services. This categorization can be based on the amount of spending in some periodic time after acquisition or the product type that the customer spent most of their order amount in some period of time.

# 1 <span style='color:#DC33FF'>|</span> Import Libraries</b>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import os
import openpyxl
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse

# <b>2 <span style='color:#DC33FF'>|</span> Reading Data</b>

In [ ]:
df = pd.read_excel("HotelCustomersDataset.xlsx")

# <b>3 <span style='color:#DC33FF'>|</span> Explore Data & Data Preprocessing</b>

It is displayed the first 10 rows of dataset.

In [ ]:
df.head(10)

It is displayed all the rows of dataset.

In [ ]:
display(df)

The dataset contains 83590 instances (customers) and 31 variables. 

In [ ]:
df.shape

The different variables are the following ones;

In [ ]:
df.columns

There are a total of 80 duplicated instances.

In [ ]:
df.duplicated(subset=['Nationality', 'Age', 'DaysSinceCreation', 'NameHash',
       'DocIDHash', 'AverageLeadTime', 'LodgingRevenue', 'OtherRevenue',
       'BookingsCanceled', 'BookingsNoShowed', 'BookingsCheckedIn',
       'PersonsNights', 'RoomNights', 'DaysSinceLastStay',
       'DaysSinceFirstStay', 'DistributionChannel', 'MarketSegment',
       'SRHighFloor', 'SRLowFloor', 'SRAccessibleRoom', 'SRMediumFloor',
       'SRBathtub', 'SRShower', 'SRCrib', 'SRKingSizeBed', 'SRTwinBed',
       'SRNearElevator', 'SRAwayFromElevator', 'SRNoAlcoholInMiniBar',
       'SRQuietRoom']).sum()

In [ ]:
df[df.duplicated(subset=['Nationality', 'Age', 'DaysSinceCreation', 'NameHash',
       'DocIDHash', 'AverageLeadTime', 'LodgingRevenue', 'OtherRevenue',
       'BookingsCanceled', 'BookingsNoShowed', 'BookingsCheckedIn',
       'PersonsNights', 'RoomNights', 'DaysSinceLastStay',
       'DaysSinceFirstStay', 'DistributionChannel', 'MarketSegment',
       'SRHighFloor', 'SRLowFloor', 'SRAccessibleRoom', 'SRMediumFloor',
       'SRBathtub', 'SRShower', 'SRCrib', 'SRKingSizeBed', 'SRTwinBed',
       'SRNearElevator', 'SRAwayFromElevator', 'SRNoAlcoholInMiniBar',
       'SRQuietRoom'], keep= False)]

As there are 80 duplicated instances with matching data in all categories, we will remove them because they do not provide extra information and they are redundant records.

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.shape

I will check if there are nulls.

In [ ]:
df.isna().sum()

Check which percentages are null over the totality of the data.

In [ ]:
100*df.isna().sum()/df.shape[0]

In [ ]:
plt.figure(figsize = (15,8))
sns.heatmap(df.isnull().sort_values(by="Age"))

In [ ]:
df.describe()

As the percentage of nulls is 4.5% over the total data, I will remove them, because it will not affect our data and to our analysis.

Besides, I checked that in the variable **"Age"**, minimum age is -11 years old. Definition: Customer's age (in years) at the last day of the extraction period. So, I will proceed to eliminate not only the nulls registers but also those whose value is negative.

In [ ]:
df=df.dropna(subset=['Age'])

In [ ]:
df.isna().sum()

In [ ]:
df = df[df['Age'] > 0]

In [ ]:
df.describe()

On the other hand, there is another inconsistent data. The minimum **"AvergagedLeadTime"** is -1, so it has not sense for this variable. Definition: The average number of days elapsed between the customer's booking date and arrival date. In other words, this variable is calculated by dividing the sum of the number of days elapsed between the moment each booking was made and its arrival date, by the total of bookings made by the customer.

I will proceed to eliminate those negative registers.

In [ ]:
df = df[df['AverageLeadTime'] >= 0] 

In [ ]:
df.describe()

Ordeno la tabla por DaysSinceCreation.

In [ ]:
df.groupby("DaysSinceCreation", group_keys=True).apply(lambda x: x)

In [ ]:
df.nunique()

Elimino las features "ID", "NameHash" y "DocIDHash", ya que no son importantes para el estudio del modelo de ML.

In [ ]:
df1 = df.drop(['ID', 'NameHash', 'DocIDHash'], axis=1)

In [ ]:
df1.head(15)

In [ ]:
df1.dtypes

There are 187 source markets, with the top 20 markets are included more than 90% of the instances (customers). In order to realize a one hot encoder it is necessary to reduce the dimensionality of the database with the less important markets.

In [ ]:
df1['Nationality'].nunique()

In [ ]:
top_20_markets = df1['Nationality'].value_counts().head(20)
names = top_20_markets.keys()

In [ ]:
top_20_markets

In [ ]:
names

In [ ]:
top_20_markets.sum()

In [ ]:
df1['Nationality'].value_counts().sum()

In [ ]:
perc_top_20_markets = (top_20_markets.sum() / df1['Nationality'].value_counts().sum())*100

In [ ]:
perc_top_20_markets 

In [ ]:
plt.figure(figsize=(20,15))
plt.pie(top_20_markets, labels=names, autopct="%0.1f %%")
plt.show()

In [ ]:
num_of_countries = df['Nationality'].nunique()
print('The number of source markets before preprocessing is %s' %num_of_countries)

In [ ]:
df1.Nationality

In [ ]:
df1 = df1[df1['Age'] < 100]
df1

In [ ]:
names_top20 = df1.groupby('Nationality').count().sort_values(by = 'Age', ascending = False)
top_20_source_market = list(names_top20[names_top20['Age'] < 724]['Age'].keys())

for i in top_20_source_market:
    df1['Nationality'] = df1['Nationality'].str.replace(i,'')
    
print('The number of top source markets after deleting those with less than 724 instances is: %s' %df1['Nationality'].nunique())

I create new features, by combining the existing features given in the dataset;

* **Total Revenue** It is the addition of LodingRevenue and OtherRevenue.
* **ADR (Average Daily Rate)** A hotel's average daily rate (ADR) is the average price guests pay for rooms. It can be measured on a given night or over a specific period of time, such as a month or a year.
* **ALOS (Average length of stay)** refers to the number of nights guests stay on average over a given period of time, usually a month or a year.

In [ ]:
df1['TotalRevenue'] = df1['LodgingRevenue'] + df1['OtherRevenue']

In [ ]:
df1.groupby('TotalRevenue')['Nationality'].unique().str[0]

In [ ]:
df1["ADR"] = df1["LodgingRevenue"]/df1["RoomNights"]

In [ ]:
df1["ADR"]

In [ ]:
df1["ALOS"]= df1["RoomNights"]/df1["BookingsCheckedIn"]

In [ ]:
df1["ALOS"]

**I delete the nulls of columns "ADR" & "ALOS"**;

df1=df1.dropna(subset=['ADR','ALOS'])

df1.dtypes

In [ ]:
display(df1)

In [ ]:
df1.isna().sum()

df1.dropna()

# <b>4 <span style='color:#DC33FF'>|</span> OneHotEncoding</b>

En este dataset, las características categóricas son las correspondientes a los tipos "object". Extraemos la lista de estas características usando el método select_dtypes de pandas.

In [ ]:
cat_features = df1.select_dtypes(include = ["object"]).columns
cat_features

In [ ]:
cat_df1 = pd.get_dummies(df1[cat_features], drop_first = True)
cat_df1.head()

* drop_first = True, que elimina la primera de las columnas generadas para cada característica con el objetivo de evitar la colinealidad (que una de las columnas sea combinación lineal de las demás)

Ahora, basta eliminar las características originales:

In [ ]:
df1.drop(cat_features, axis = 1, inplace = True)

Y procecemos a concatenar el dataframe resultante con el dataframe que hemos generado con las características "dummies":

In [ ]:
df1 = pd.concat([df1, cat_df1], axis = 1)

In [ ]:
print(df1.columns.tolist())
type(df1.columns.tolist())

In [ ]:
df1.dtypes

imputer = KNNImputer(n_neighbors=1, weights="distance")
df1 = pd.DataFrame(imputer.fit_transform(df1), columns = ['Age', 'DaysSinceCreation', 'AverageLeadTime', 'LodgingRevenue',
       'OtherRevenue', 'BookingsCanceled', 'BookingsNoShowed',
       'BookingsCheckedIn', 'PersonsNights', 'RoomNights', 'DaysSinceLastStay',
       'DaysSinceFirstStay', 'SRHighFloor', 'SRLowFloor', 'SRAccessibleRoom',
       'SRMediumFloor', 'SRBathtub', 'SRShower', 'SRCrib', 'SRKingSizeBed',
       'SRTwinBed', 'SRNearElevator', 'SRAwayFromElevator',
       'SRNoAlcoholInMiniBar', 'SRQuietRoom', 'TotalRevenue', 'ADR', 'ALOS',
       'Nationality_AUT', 'Nationality_BEL', 'Nationality_BRA',
       'Nationality_CAN', 'Nationality_CHE', 'Nationality_CHN',
       'Nationality_DEU', 'Nationality_ESP', 'Nationality_FRA',
       'Nationality_GBR', 'Nationality_IRL', 'Nationality_ISR',
       'Nationality_ITA', 'Nationality_NLD', 'Nationality_NOR',
       'Nationality_POL', 'Nationality_PRT', 'Nationality_SWE',
       'Nationality_USA', 'DistributionChannel_Direct',
       'DistributionChannel_Electronic Distribution',
       'DistributionChannel_Travel Agent/Operator',
       'MarketSegment_Complementary', 'MarketSegment_Corporate',
       'MarketSegment_Direct', 'MarketSegment_Groups', 'MarketSegment_Other',
       'MarketSegment_Travel Agent/Operator'])
df1

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy ="most_frequent")
imp.fit(df1)
df1 = pd.DataFrame(imp.transform(df1), columns = ['Age', 'DaysSinceCreation', 'AverageLeadTime', 'LodgingRevenue',
       'OtherRevenue', 'BookingsCanceled', 'BookingsNoShowed',
       'BookingsCheckedIn', 'PersonsNights', 'RoomNights', 'DaysSinceLastStay',
       'DaysSinceFirstStay', 'SRHighFloor', 'SRLowFloor', 'SRAccessibleRoom',
       'SRMediumFloor', 'SRBathtub', 'SRShower', 'SRCrib', 'SRKingSizeBed',
       'SRTwinBed', 'SRNearElevator', 'SRAwayFromElevator',
       'SRNoAlcoholInMiniBar', 'SRQuietRoom', 'TotalRevenue', 'ADR', 'ALOS',
       'Nationality_AUT', 'Nationality_BEL', 'Nationality_BRA',
       'Nationality_CAN', 'Nationality_CHE', 'Nationality_CHN',
       'Nationality_DEU', 'Nationality_ESP', 'Nationality_FRA',
       'Nationality_GBR', 'Nationality_IRL', 'Nationality_ISR',
       'Nationality_ITA', 'Nationality_NLD', 'Nationality_NOR',
       'Nationality_POL', 'Nationality_PRT', 'Nationality_SWE',
       'Nationality_USA', 'DistributionChannel_Direct',
       'DistributionChannel_Electronic Distribution',
       'DistributionChannel_Travel Agent/Operator',
       'MarketSegment_Complementary', 'MarketSegment_Corporate',
       'MarketSegment_Direct', 'MarketSegment_Groups', 'MarketSegment_Other',
       'MarketSegment_Travel Agent/Operator'])
df1

In [ ]:
df.isna().sum()

# <b>5 <span style='color:#DC33FF'>|</span> Defining X, y</b>

In [ ]:
y = df1["AverageLeadTime"]
X = df1.drop(columns=["AverageLeadTime"])

In [ ]:
X

In [ ]:
y

In [ ]:
sns.pairplot(pd.concat([X, y], axis=1))

# <b>6 <span style='color:#DC33FF'>|</span> Train Test Split</b>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train

In [ ]:
y_train

# <b>7 <span style='color:#DC33FF'>|</span> Trying different models</b>

In [ ]:
models = {
    "LinearRegression":LinearRegression(),
    "Poly_2":Pipeline([("poly_features", PolynomialFeatures(degree=2)),
                       ("linear_regression", LinearRegression())]), 
    "Lasso": Lasso(alpha=1),
    "Ridge": Ridge(alpha=1),
    "Support Vector Machine": SVR(),
    "SGD": SGDRegressor(max_iter=10000),
    "Random Forest": RandomForestRegressor(),
}

In [ ]:
results_train = {}
results_test = {}

for name, model in models.items():
    print(f"Training {name}")
    model.fit(X_train, y_train)
    results_train[name] = mse(y_train, model.predict(X_train))**5
    results_test[name] = mse(y_test, model.predict(X_test))**5

In [ ]:
results_train

In [ ]:
results_test

models = {
    "LinearRegression":LinearRegression(),
    "Poly_2":Pipeline([("poly_features", PolynomialFeatures(degree=2)),
                       ("linear_regression", LinearRegression())]),
    "Poly_3":Pipeline([("poly_features", PolynomialFeatures(degree=3)),
                       ("linear_regression", LinearRegression())]),   
    "Poly_4":Pipeline([("poly_features", PolynomialFeatures(degree=4)),
                       ("linear_regression", LinearRegression())]),     
    "Poly_5":Pipeline([("poly_features", PolynomialFeatures(degree=5)),
                       ("linear_regression", LinearRegression())]),
    "Lasso": Lasso(alpha=1),
    "Ridge": Ridge(alpha=1),
    "Support Vector Machine": SVR(),
    "SGD": SGDRegressor(max_iter=10000),
    "Random Forrest": RandomForestRegressor(),
    "Polynomial_dg_2_Lasso": Pipeline([("poly", PolynomialFeatures(degree=3)),
                                      ("Lasso", Lasso (alpha=10))]),  
}

results_train = {}
results_test = {}

for name, model in models.items():
    print(f"Training {name}")
    model.fit(X_train, y_train)
    results_train[name] = mse(y_train, model.predict(X_train))**5
    results_test[name] = mse(y_test, model.predict(X_test))**5

results_train

results_test

In [324]:
X_train

,Nationality,Age,DaysSinceCreation,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,...,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
29254,FRA,50.0,582,972.00,99.0,0,0,1,6,3,...,0,0,0,0,0,0,0,0,0,0
6226,FRA,39.0,962,992.00,78.0,0,0,1,2,2,...,0,0,0,0,0,0,0,0,0,0
56586,CHE,11.0,225,0.00,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51960,BRA,66.0,264,384.48,63.0,0,0,1,9,3,...,0,0,0,0,1,0,0,0,0,0
59845,FRA,47.0,199,238.00,14.0,0,0,1,2,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75104,IRL,28.0,86,0.00,0.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29681,BEL,44.0,575,345.60,42.0,0,0,1,6,2,...,0,0,0,0,0,1,0,0,0,0
14751,GBR,30.0,831,358.00,51.5,0,0,1,2,2,...,0,0,0,0,0,0,0,0,0,0
55835,EST,28.0,232,0.00,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [303]:
from sklearn. preprocessing import OneHotEncoder

In [304]:
X_train = pd.get_dummies(X_train, prefix = "", prefix_sep="")

In [305]:
X_train.head()

,Age,DaysSinceCreation,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,...,Direct,Electronic Distribution,Travel Agent/Operator,Aviation,Complementary,Corporate,Direct,Groups,Other,Travel Agent/Operator
75032,70.0,87,0.0,0.0,0,0,0,0,0,-1,...,1,0,0,0,0,0,1,0,0,0
42024,34.0,390,74.0,7.0,0,0,1,1,1,391,...,0,0,1,0,0,0,0,1,0,0
24209,55.0,655,241.3,56.0,0,0,1,8,4,659,...,0,0,1,0,0,0,0,0,1,0
35068,36.0,494,436.0,31.0,0,0,1,4,4,498,...,0,0,1,0,0,0,0,0,0,1
75293,60.0,85,0.0,0.0,0,0,0,0,0,-1,...,0,0,1,0,0,0,0,0,1,0


In [306]:
X_train.shape

(63794, 216)

In [307]:
ohe = OneHotEncoder(handle_unknown='ignore')

In [308]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [309]:
X_train["Nationality"]

29254    FRA
6226     FRA
56586    CHE
51960    BRA
59845    FRA
        ... 
75104    IRL
29681    BEL
14751    GBR
55835    EST
51105    GBR
Name: Nationality, Length: 59807, dtype: object

In [310]:
X_train["DistributionChannel"]

29254    Travel Agent/Operator
6226                    Direct
56586                   Direct
51960    Travel Agent/Operator
59845    Travel Agent/Operator
                 ...          
75104    Travel Agent/Operator
29681    Travel Agent/Operator
14751                Corporate
55835    Travel Agent/Operator
51105    Travel Agent/Operator
Name: DistributionChannel, Length: 59807, dtype: object

In [311]:
X_train["MarketSegment"]

29254        Other
6226        Direct
56586       Direct
51960        Other
59845       Groups
           ...    
75104        Other
29681        Other
14751    Corporate
55835       Groups
51105        Other
Name: MarketSegment, Length: 59807, dtype: object

In [312]:
type(X_train[["Nationality"]])

pandas.core.frame.DataFrame

In [313]:
ohe.fit(X_train[["DistributionChannel"]])

OneHotEncoder(handle_unknown='ignore')

In [314]:
ohe.fit(X_train[["MarketSegment"]])

OneHotEncoder(handle_unknown='ignore')

In [315]:
ohe.fit(X_train[["Nationality"]])

OneHotEncoder(handle_unknown='ignore')

In [316]:
ohe.categories_

[array(['ABW', 'AGO', 'AIA', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ASM',
        'ATA', 'ATF', 'ATG', 'AUS', 'AUT', 'AZE', 'BEL', 'BEN', 'BGD',
        'BGR', 'BHR', 'BHS', 'BIH', 'BLR', 'BOL', 'BRA', 'BRB', 'BWA',
        'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD', 'COL',
        'COM', 'CPV', 'CRI', 'CUB', 'CYM', 'CYP', 'CZE', 'DEU', 'DMA',
        'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH',
        'FIN', 'FRA', 'FRO', 'GAB', 'GBR', 'GEO', 'GHA', 'GIB', 'GIN',
        'GNB', 'GNQ', 'GRC', 'GTM', 'GUY', 'HKG', 'HRV', 'HTI', 'HUN',
        'IDN', 'IND', 'IOT', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR', 'ITA',
        'JAM', 'JEY', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KOR', 'KWT',
        'LAO', 'LBN', 'LBY', 'LCA', 'LIE', 'LKA', 'LTU', 'LUX', 'LVA',
        'MAR', 'MCO', 'MDG', 'MDV', 'MEX', 'MKD', 'MLI', 'MLT', 'MMR',
        'MNE', 'MOZ', 'MUS', 'MWI', 'MYS', 'NAM', 'NCL', 'NGA', 'NIC',
        'NLD', 'NOR', 'NPL', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL',
      

In [317]:
encoded_Nationality_train = ohe.transform(X_train[["Nationality"]])

In [321]:
encoded_Nationality_train.todense()[:2]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [319]:
encoded_DistributionChannel_train = ohe.transform(X_train[["DistributionChannel"]])

/home/iremirezdeganuza/miniconda3/envs/cohortes/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- DistributionChannel
Feature names seen at fit time, yet now missing:
- Nationality

  warnings.warn(message, FutureWarning)


In [322]:
encoded_DistributionChannel_train.todense()[:2]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [320]:
encoded_MarketSegment_train = ohe.transform(X_train[["MarketSegment"]])

/home/iremirezdeganuza/miniconda3/envs/cohortes/lib/python3.10/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- MarketSegment
Feature names seen at fit time, yet now missing:
- Nationality

  warnings.warn(message, FutureWarning)


In [264]:
encoded_MarketSegment_train.todense()[:2]

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [240]:
encoded_Nationality_train = pd.DataFrame(encoded_Nationality_train, index=X_train.index)
pd.concat([X_train, encoded_Nationality_train], axis=1)

,Nationality,Age,DaysSinceCreation,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,...,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom,0
38532,NLD,53.0,446,1428.0,112.0,0,0,1,7,7,...,0,0,0,0,0,0,0,0,0,"(0, 117)\t1.0"
71057,AUT,58.0,116,755.0,255.5,0,0,1,10,5,...,0,0,0,1,0,0,0,0,0,"(0, 13)\t1.0"
59349,DEU,48.0,204,288.0,14.0,0,0,1,2,2,...,0,0,0,0,0,0,0,0,0,"(0, 43)\t1.0"
68802,GBR,8.0,132,0.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"(0, 60)\t1.0"
47381,PRT,63.0,305,109.0,34.5,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,"(0, 131)\t1.0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32825,BRA,65.0,529,120.0,45.5,0,0,1,2,1,...,0,0,0,1,0,0,0,0,0,"(0, 24)\t1.0"
32451,DEU,80.0,534,139.0,54.0,0,0,1,2,2,...,0,0,0,0,0,0,0,0,0,"(0, 43)\t1.0"
24404,FRA,48.0,652,79.2,22.0,0,0,1,2,1,...,0,0,0,0,0,0,0,0,0,"(0, 57)\t1.0"
41974,DEU,63.0,392,198.0,4.0,0,0,1,4,2,...,0,0,0,0,1,0,0,0,0,"(0, 43)\t1.0"


In [241]:
Nationality_ohe_test = ohe.transform(X_test[["Nationality"]]).todense()
Nationality_ohe_test = pd.DataFrame(Nationality_ohe_test, index=X_test.index)
pd.concat([X_test, Nationality_ohe_test], axis=1).head()

,Nationality,Age,DaysSinceCreation,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,...,165,166,167,168,169,170,171,172,173,174
31946,CHE,33.0,542,126.00,14.0,0,0,1,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80806,PRT,5.0,36,0.00,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2701,PRT,64.0,1025,318.75,292.5,0,0,1,10,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
658,FRA,55.0,1071,138.00,126.5,0,0,1,6,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77138,USA,71.0,72,411.00,129.5,0,0,1,6,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [242]:
pd.get_dummies(X_test.iloc[:5])

,Age,DaysSinceCreation,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,...,SRQuietRoom,Nationality_CHE,Nationality_FRA,Nationality_PRT,Nationality_USA,DistributionChannel_Direct,DistributionChannel_Travel Agent/Operator,MarketSegment_Direct,MarketSegment_Other,MarketSegment_Travel Agent/Operator
31946,33.0,542,126.00,14.0,0,0,1,2,1,543,...,0,1,0,0,0,0,1,0,1,0
80806,5.0,36,0.00,0.0,0,0,0,0,0,-1,...,0,0,0,1,0,1,0,1,0,0
2701,64.0,1025,318.75,292.5,0,0,1,10,5,1030,...,0,0,0,1,0,0,1,0,1,0
658,55.0,1071,138.00,126.5,0,0,1,6,3,1074,...,0,0,1,0,0,0,1,0,0,1
77138,71.0,72,411.00,129.5,0,0,1,6,3,75,...,1,0,0,0,1,0,1,0,1,0


In [213]:
ohe.transform(X_test.iloc[:2][["Nationality"]]).todense()

matrix([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 

It is important to use the same instance of encoder. We will use pd.get_dummies.

In [244]:
X_dummies = pd.get_dummies(X, columns=["Nationality", "DistributionChannel","MarketSegment"])

In [245]:
X_dummies

,Age,DaysSinceCreation,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,...,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator,MarketSegment_Aviation,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Groups,MarketSegment_Other,MarketSegment_Travel Agent/Operator
0,51.0,150,371.0,105.3,1,0,3,8,5,151,...,0,0,0,0,0,1,0,0,0,0
2,31.0,1095,0.0,0.0,0,0,0,0,0,-1,...,0,0,1,0,0,0,0,0,0,1
3,60.0,1095,240.0,60.0,0,0,1,10,5,1100,...,0,0,1,0,0,0,0,0,0,1
4,51.0,1095,0.0,0.0,0,0,0,0,0,-1,...,0,0,1,0,0,0,0,0,0,1
5,54.0,1095,230.0,24.0,0,0,1,4,2,1097,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83585,54.0,0,0.0,0.0,0,0,0,0,0,-1,...,0,0,1,0,0,0,0,0,1,0
83586,47.0,0,327.1,277.5,0,0,1,6,3,3,...,0,0,1,0,0,0,0,0,1,0
83587,53.0,0,0.0,0.0,0,0,0,0,0,-1,...,0,0,1,0,0,0,0,0,1,0
83588,85.0,0,981.3,9.0,0,0,1,6,3,3,...,0,0,1,0,0,0,0,0,1,0


In [219]:
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y)

In [220]:
X_train.shape

(59807, 213)

In [223]:
y_test.iloc[1:5]

74355     0
21750    66
40898    12
50326     0
Name: AverageLeadTime, dtype: int64

In [225]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression

In [227]:
lr = LinearRegression()

In [229]:
lr.fit(X_train, y_train)

ValueError: could not convert string to float: 'Travel Agent/Operator'

# <b>4 <span style='color:#fff700ba'>|</span> KMeans</b>